## Resolve dependências

In [4]:
!pip install -q albumentations==1.4.6 opencv-python pycocotools tqdm
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q pycocotools albumentations==1.4.6 opencv-python tqdm
!pip uninstall -y sympy
!pip install sympy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 9.8 MB/s eta 0:00:00
Found existing installation: sympy 1.13.3
Uninstalling sympy-1.13.3:
  Successfully uninstalled sympy-1.13.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 79.8 MB/s eta 0:00:00


In [8]:
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.24.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 694356, done.
remote: Counting objects: 100% (1691/1691), done.
remote: Compressing objects: 100% (879/879), done.
remote: Total 694356 (delta 1519), reused 885 (delta 804), pack-reused 692665 (from 3)
Receiving objects: 100% (694356/694356), 1.22 GiB | 37.27 MiB/s, done.
Resolving deltas: 100% (650252/650252), done.
fatal: not a git repository (or any of the parent directories): .git


In [3]:
import sys
# sys.path.append("/content/drive/MyDrive/TCC/src/")
sys.path.append("/content/drive/MyDrive/TCC/notebooks/Faster R-CNN")

In [1]:
from pathlib import Path

DATASET_ROOT = Path("/content/drive/MyDrive/TCC/DATASET")
paths = {
    "bh_watertanks": {
        "images": {
            "train": DATASET_ROOT/"bh_watertanks/images/train",
            "val":   DATASET_ROOT/"bh_watertanks/images/val",
            "test":  DATASET_ROOT/"bh_watertanks/images/test",
        },
        "ann": {
            "train": DATASET_ROOT/"bh_watertanks/annotations/instances_train.json",
            "val":   DATASET_ROOT/"bh_watertanks/annotations/instances_val.json",
            "test":  DATASET_ROOT/"bh_watertanks/annotations/instances_test.json",
        }
    }
}

## Carrega Loaders

In [5]:
from my_utils.coco_dataset import CocoDetDataset
from my_utils.my_utils import get_transforms, collate_fn, evaluate_and_visualize
from torch.utils.data import Dataset, DataLoader


watertanks_train = CocoDetDataset(
    paths["bh_watertanks"]["images"]["train"],
    paths["bh_watertanks"]["ann"]["train"],
    transforms=get_transforms(train=True))

watertanks_val   = CocoDetDataset(
    paths["bh_watertanks"]["images"]["val"],
    paths["bh_watertanks"]["ann"]["val"],
    transforms=get_transforms(train=False))

watertanks_test  = CocoDetDataset(
    paths["bh_watertanks"]["images"]["test"],
    paths["bh_watertanks"]["ann"]["test"],
    transforms=get_transforms(train=False))


watertanks_train_loader = DataLoader(watertanks_train, batch_size=2, shuffle=True,  num_workers=2, collate_fn=collate_fn, pin_memory=True)
watertanks_val_loader   = DataLoader(watertanks_val,   batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)
watertanks_test_loader  = DataLoader(watertanks_test,  batch_size=2, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

loading annotations into memory...
Done (t=2.39s)
creating index...
index created!
loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
loading annotations into memory...
Done (t=0.63s)
creating index...
index created!


## Inicializa model

In [6]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 1 + 2

weights = torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1
model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=weights)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth


100%|██████████| 167M/167M [00:00<00:00, 229MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [ ]:
checkpoint_path = '/content/drive/MyDrive/TCC/checkpoints/fasterrcnn_pools_focused.pth'
state = torch.load(checkpoint_path)
model.load_state_dict(state)

<All keys matched successfully>

## Treina modelo

In [9]:
from torch.cuda.amp import GradScaler
scaler = GradScaler()

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.002,
    momentum=0.9,
    weight_decay=5e-4
)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=10,
    eta_min=1e-5
)


/tmp/ipython-input-1300750638.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [11]:
from engine import train_one_epoch
import torch

epochs = 10

for epoch in range(epochs):
  for p in model.backbone.body.parameters():
    p.requires_grad = True
    new_params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(new_params, lr=0.002, momentum=0.9, weight_decay=5e-4)

  train_one_epoch(model, optimizer, watertanks_train_loader, device, epoch, print_freq=10)
  lr_scheduler.step()



/content/engine.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler is not None):


Epoch: [0]  [ 0/50]  eta: 0:04:30  lr: 0.000043  loss: 8.3607 (8.3607)  loss_classifier: 1.1048 (1.1048)  loss_box_reg: 0.0575 (0.0575)  loss_objectness: 5.3553 (5.3553)  loss_rpn_box_reg: 1.8430 (1.8430)  time: 5.4161  data: 2.6449  max mem: 3555
Epoch: [0]  [10/50]  eta: 0:00:49  lr: 0.000451  loss: 6.4891 (6.3709)  loss_classifier: 1.0214 (0.9790)  loss_box_reg: 0.0718 (0.0703)  loss_objectness: 3.7792 (3.6942)  loss_rpn_box_reg: 1.6189 (1.6275)  time: 1.2381  data: 0.7378  max mem: 4487
Epoch: [0]  [20/50]  eta: 0:00:30  lr: 0.000858  loss: 3.2416 (4.3938)  loss_classifier: 0.6068 (0.7349)  loss_box_reg: 0.0669 (0.0667)  loss_objectness: 1.1607 (2.2004)  loss_rpn_box_reg: 1.3413 (1.3918)  time: 0.8043  data: 0.5311  max mem: 4542
Epoch: [0]  [30/50]  eta: 0:00:19  lr: 0.001266  loss: 1.6986 (3.3962)  loss_classifier: 0.4173 (0.6081)  loss_box_reg: 0.0635 (0.0745)  loss_objectness: 0.3340 (1.5926)  loss_rpn_box_reg: 0.7870 (1.1209)  time: 0.8204  data: 0.5459  max mem: 4577
Epoch: [

/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch: [1]  [ 0/50]  eta: 0:00:42  lr: 0.002000  loss: 0.7888 (0.7888)  loss_classifier: 0.1544 (0.1544)  loss_box_reg: 0.1234 (0.1234)  loss_objectness: 0.2800 (0.2800)  loss_rpn_box_reg: 0.2310 (0.2310)  time: 0.8477  data: 0.5632  max mem: 4577
Epoch: [1]  [10/50]  eta: 0:00:13  lr: 0.002000  loss: 0.7546 (0.7634)  loss_classifier: 0.1836 (0.1989)  loss_box_reg: 0.1051 (0.1059)  loss_objectness: 0.2558 (0.2444)  loss_rpn_box_reg: 0.2310 (0.2143)  time: 0.3325  data: 0.0516  max mem: 4577
Epoch: [1]  [20/50]  eta: 0:00:09  lr: 0.002000  loss: 0.7927 (0.7855)  loss_classifier: 0.2008 (0.2030)  loss_box_reg: 0.1103 (0.1116)  loss_objectness: 0.2521 (0.2483)  loss_rpn_box_reg: 0.2311 (0.2226)  time: 0.2793  data: 0.0004  max mem: 4577
Epoch: [1]  [30/50]  eta: 0:00:05  lr: 0.002000  loss: 0.8119 (0.7930)  loss_classifier: 0.2036 (0.2029)  loss_box_reg: 0.1226 (0.1248)  loss_objectness: 0.2521 (0.2456)  loss_rpn_box_reg: 0.2171 (0.2197)  time: 0.2788  data: 0.0003  max mem: 4599
Epoch: [

## Avalia Modelo

In [13]:
from my_utils.my_utils import evaluate_ap_multi_iou
import numpy as np

csv_path = "/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/results.csv"
df_results = evaluate_ap_multi_iou(
    model=model,
    loader=watertanks_test_loader,
    device=device,
    score_thr=0.25,
    iou_list=(0.50, 0.75, 0.90),
    iou_grid=np.arange(0.50, 0.96, 0.05),
    out_csv_path=csv_path
)

==== Resultados por classe ====
Classe 1 | mAP[.50:.95]=0.000 | AP@0.50=0.000 | AP@0.75=0.000 | AP@0.90=0.000 (GT=2723)

📊 CSV salvo em: /content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/results.csv


In [20]:
torch.save(model.state_dict(), '/content/drive/MyDrive/TCC/notebooks/Faster R-CNN/watertanks/model/wt_t_.pth')

In [15]:
from my_utils.my_utils import show_predictions

show_predictions(
    model=model,
    loader=watertanks_test_loader,
    device=device,
    n=2,
    score_thr=0.25
)


Output hidden; open in https://colab.research.google.com to view.